In [1]:
import alpaca_trade_api as tradeapi
import pandas as pd
from pathlib import Path
import numpy as np
import datetime as dt
# import seaborn as sns
from pathlib import Path
import os
from datetime import datetime, timedelta
# import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
np.random.seed(42)

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY_ID")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Initialize the API
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [90]:
comm_feb_csv = Path('comm_feb.csv')
fin_feb_csv = Path('fin_feb.csv')
food_feb_csv = Path('food_feb.csv')
hedge_feb_csv = Path('hedge_feb.csv')
pharma_feb_csv = Path('pharma_feb.csv')
sp500_feb_csv = Path('sp500_feb.csv')

comm_apr_csv = Path('comm_apr.csv')
fin_apr_csv = Path('fin_apr.csv')
food_apr_csv = Path('food_apr.csv')
hedge_apr_csv = Path('hedge_apr.csv')
pharma_apr_csv = Path('pharma_apr.csv')
sp500_apr_csv = Path('sp500_apr.csv')


comm_feb_df = pd.read_csv(comm_feb_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
fin_feb_df = pd.read_csv(fin_feb_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
food_feb_df = pd.read_csv(food_feb_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
hedge_feb_df = pd.read_csv(hedge_feb_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
pharma_feb_df = pd.read_csv(hedge_feb_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp500_feb_df = pd.read_csv(sp500_feb_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)

comm_apr_df = pd.read_csv(comm_apr_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
fin_apr_df = pd.read_csv(fin_apr_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
food_apr_df = pd.read_csv(food_apr_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
hedge_apr_df = pd.read_csv(hedge_apr_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
pharma_apr_df = pd.read_csv(hedge_apr_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp500_apr_df = pd.read_csv(sp500_apr_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)

# Change SP500 field 'close' to 'SP500' so it's easier to read in the combined DF
sp500_feb_df.rename(columns={'Close': 'SP500'}, inplace=True)
sp500_apr_df.rename(columns={'Close': 'SP500'}, inplace=True)

In [91]:
##Daily Returns
comm_feb_returns = comm_feb_df.pct_change()
fin_feb_returns = fin_feb_df.pct_change()
food_feb_returns = food_feb_df.pct_change()
hedge_feb_returns = hedge_feb_df.pct_change()
pharma_feb_returns = pharma_feb_df.pct_change()
sp500_feb_returns = sp500_feb_df.pct_change()

comm_apr_returns = comm_apr_df.pct_change()
fin_apr_returns = fin_apr_df.pct_change()
food_apr_returns = food_apr_df.pct_change()
hedge_apr_returns = hedge_apr_df.pct_change()
pharma_apr_returns = pharma_apr_df.pct_change()
sp500_apr_returns = sp500_apr_df.pct_change()

#Drop nulls
comm_feb_ret = comm_feb_returns.dropna()
fin_feb_ret = fin_feb_returns.dropna()
food_feb_ret = food_feb_returns.dropna()
hedge_feb_ret = hedge_feb_returns.dropna()
pharma_feb_ret = pharma_feb_returns.dropna()
sp500_feb_ret = sp500_feb_returns.dropna()

comm_apr_ret = comm_apr_returns.dropna()
fin_apr_ret = fin_apr_returns.dropna()
food_apr_ret = food_apr_returns.dropna()
hedge_apr_ret = hedge_apr_returns.dropna()
pharma_apr_ret = pharma_apr_returns.dropna()
sp500_apr_ret = sp500_apr_returns.dropna()

comm_feb_ret = comm_feb_ret.loc[comm_feb_ret.index.dropna()]
fin_feb_ret = fin_feb_ret.loc[fin_feb_ret.index.dropna()]
food_feb_ret = food_feb_ret.loc[food_feb_ret.index.dropna()]
hedge_feb_ret = hedge_feb_ret.loc[hedge_feb_ret.index.dropna()]
pharma_feb_ret = pharma_feb_ret.loc[pharma_feb_ret.index.dropna()]
sp500_feb_ret = sp500_feb_ret.loc[sp500_feb_ret.index.dropna()]

comm_apr_ret = comm_apr_ret.loc[comm_apr_ret.index.dropna()]
fin_apr_ret = fin_apr_ret.loc[fin_apr_ret.index.dropna()]
food_apr_ret = food_apr_ret.loc[food_apr_ret.index.dropna()]
hedge_apr_ret = hedge_apr_ret.loc[hedge_apr_ret.index.dropna()]
pharma_apr_ret = pharma_apr_ret.loc[pharma_apr_ret.index.dropna()]
sp500_apr_ret = sp500_apr_ret.loc[sp500_apr_ret.index.dropna()]

In [96]:
#Combined Pre-COVID-19 ('feb') portfolios and  Post-COVID-19 ('apr') portfolios 
combined_feb_df = pd.concat([comm_feb_ret, fin_feb_ret, food_feb_ret, hedge_feb_ret, pharma_feb_ret, sp500_feb_ret], axis="columns", join="inner")
# combined_apr_df = pd.merge([comm_apr_ret, fin_apr_ret, food_apr_ret, hedge_apr_ret, pharma_apr_ret, sp500_apr_ret], axis="columns", join="inner")
combined_apr_df = pd.concat([comm_apr_ret, fin_apr_ret, food_apr_ret, hedge_apr_ret, pharma_apr_ret, sp500_apr_ret], axis="columns", join="inner")

In [98]:
len(combined_apr_df)

301

In [ ]:
def analyze_portfolio(portfolio):
    '''
    Since we decided to work with data from Google Finance (instead of Alpaca),
    
    `portfolio` := list of stock CSVs
    
    Returns: TODO
    '''
    for stock in portfolio:
        stock_returns = stock.pct_change()
        stock_returns = stock_returns.dropna()
        
        # In case there are any NaT in the index
        stock_returns = stock_returns.loc[stock_returns.index.dropna()]
        
        stock_std = stock_returns.std()
        
        

In [73]:
df1 = pd.DataFrame({'a': [1,2,3], 'b': [1,2,3]})
df2 = pd.DataFrame({'c': [1,2], 'd': [1,2]})

pd.concat([df1, df2], axis=1, join='inner')
# pd.concat?

,a,b,c,d
0,1,1,1,1
1,2,2,2,2


In [82]:
sp500_feb_df

,Close
Date,
2019-02-19,2779.76
2019-02-20,2784.70
2019-02-21,2774.88
2019-02-22,2792.67
2019-02-25,2796.11
2019-02-26,2793.90
2019-02-27,2792.38
2019-02-28,2784.49
2019-03-01,2803.69
